---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler

def blight_model():
    
    # Load data files
    addresses = pd.read_csv('readonly/addresses.csv')
    latlons = pd.read_csv('readonly/latlons.csv')
    Test = pd.read_csv('readonly/test.csv')
    Train = pd.read_csv('readonly/train.csv', encoding='ISO-8859-1', low_memory=False)
    
    # Removing compliance = Null
    Train = Train[(Train['compliance'] == 0) | (Train['compliance'] == 1)]
    
    # Avoiding Data Leakage, removing  features that don't exist in the test set
    Train = Train.drop(['payment_amount','payment_date','payment_status',
                        'balance_due','collection_status','compliance_detail'], axis=1)
    
    # Removing features (string and others that don't make to much sense)
    remove = ['state','inspector_name','violator_name','violation_street_number',
              'violation_street_name','violation_zip_code','mailing_address_str_number',
              'mailing_address_str_name','city','zip_code','non_us_str_code','country',
              'violation_description','disposition','discount_amount',
              'judgment_amount','clean_up_cost','grafitti_status']
    Train = Train.drop(remove,axis=1)
    Test = Test.drop(remove,axis=1)

    # Filtering days between hearing_date and ticket_issued_date
    ticket_issued_X = Train['ticket_issued_date'].str.split(' ',n=1,expand=True)
    hearing_date_X = Train['hearing_date'].str.split(' ',n=1,expand=True)
    Train['ticket_issued_date'] = ticket_issued_X[0]
    Train['hearing_date'] = hearing_date_X[0]
    Train[['ticket_issued_date','hearing_date']] = Train[['ticket_issued_date','hearing_date']].apply(pd.to_datetime)
    Train['days'] =  (Train['hearing_date'] - Train['ticket_issued_date']).dt.days

    ticket_issued_y = Test['ticket_issued_date'].str.split(' ',n=1,expand=True)
    hearing_date_y = Test['hearing_date'].str.split(' ',n=1,expand=True)
    Test['ticket_issued_date'] = ticket_issued_y[0]
    Test['hearing_date'] = hearing_date_y[0]
    Test[['ticket_issued_date','hearing_date']] = Test[['ticket_issued_date','hearing_date']].apply(pd.to_datetime)
    Test['days'] =  (Test['hearing_date'] - Test['ticket_issued_date']).dt.days
    
    Train = Train.drop(['ticket_issued_date','hearing_date'],axis=1)
    Test = Test.drop(['ticket_issued_date','hearing_date'],axis=1)
    Train = Train.drop(['agency_name','violation_code'],axis=1)
    Test = Test.drop(['agency_name','violation_code'],axis=1)
    

    # join addresses and latlons by 'address'
    local = pd.merge(addresses,latlons, on=['address'],how='inner')
    # removing 'address' column
    local = local.drop(['address'], axis=1)
    # join local to Train and Test sets
    Train = pd.merge(Train,local,on=['ticket_id'],how='inner')
    Test = pd.merge(Test,local,on=['ticket_id'],how='inner')
    
    # Replace nan values with average of columns
    Train = Train.fillna(Train.mean())
    Test = Test.fillna(Test.mean())

    # Applie MinMaxScaler on all numeric features with range 0-1
    #scaler = MinMaxScaler(feature_range=(0,1))
    #numeric = ['fine_amount','admin_fee','state_fee','late_fee',]
    #Train[numeric] = scaler.fit_transform(Train[numeric])
    #Test[numeric] = scaler.fit_transform(Test[numeric])
    
    # Split Train into X and y
    y_train = Train['compliance']
    X_train = Train.drop(['compliance'],axis=1)
    
    #print(X_train)
    #print(y_train)
    #  Machine Learning model
    parameters = {
        "loss":["deviance"],
        "max_depth":[3,5,8],
        "n_estimators":[10]
    }
    clf = GridSearchCV(GradientBoostingClassifier(),parameters,cv=5)
    clf.fit(X_train,y_train)
    
    #
    y_pred = clf.predict_proba(Test)[:,1]
    
    # Integrate with the original Test set DataFrame, then returning a 'series' type
    Original_Test = pd.read_csv('readonly/test.csv', encoding = "ISO-8859-1")
    Original_Test['compliance'] = y_pred
    Original_Test.set_index('ticket_id', inplace=True)
    
    return Original_Test.compliance 

In [6]:
blight_model()

ticket_id
284932    0.092525
285362    0.053998
285361    0.092525
285338    0.084637
285346    0.084637
            ...   
376496    0.053998
376497    0.053998
376499    0.092525
376500    0.092525
369851    0.240651
Name: compliance, Length: 61001, dtype: float64